In [1]:
from CLEAN import hogbom_clean
#from data import fits_image
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.ndimage import zoom

In [2]:
hogbom_clean?

Signature:
hogbom_clean(
    dirty_image,
    psf,
    gain=0.1,
    threshold=0.001,
    max_iterations=1000,
)
Docstring:
Performs the Hogbom CLEAN algorithm on a dirty image.

Parameters:
dirty_image (2D numpy array): The dirty image to be cleaned.
psf (2D numpy array): The point spread function (PSF) or dirty beam.
gain (float): The loop gain (fraction of the peak to subtract each iteration). Default is 0.1.
threshold (float): The stopping threshold for the peak residual. Default is 0.001.
max_iterations (int): Maximum number of iterations to perform. Default is 1000.

Returns:
clean_image (2D numpy array): The cleaned image.
residual (2D numpy array): The final residual image.
File:      ~/.conda/envs/py310/lib/python3.10/site-packages/CLEAN.py
Type:      function

In [3]:
imagename = '/safepool/claw/images/first_trecs_400sqdeg_skymodel3_sub4126warpednoiseconv2.fits'
psfname = '/safepool/claw/images/W-GC-8000chan-briggs09-big-psf_sub4126.fits'

In [4]:
imhdu = fits.open(imagename)
imhdr = imhdu[0].header
psfhdu = fits.open(psfname)
psfhdr = psfhdu[0].header
imarr = imhdu[0].data.squeeze()
psfarr = psfhdu[0].data

print(imarr.shape, psfarr.shape)

(16380, 16380) (32000, 32000)


In [13]:
im_scale = imhdr['CDELT2']*3600
psf_scale = psfhdr['CDELT2']*3600
print(f'pixel scales: {im_scale}, {psf_scale}')
psf_subsize = 257

pixel scales: 0.87890625, 0.500000000000004


In [14]:
imarr_clean, imarr_resid = hogbom_clean(imarr,
                                        zoom(psfarr[32000//2-psf_subsize//2:32000//2+psf_subsize//2,
                                                    32000//2-psf_subsize//2:32000//2+psf_subsize//2], im_scale/psf_scale),
                                        max_iterations=5, gain=0.1, threshold=1e-9)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
Gaussian restoring size of (450, 450) and width 2.972626301008067


In [15]:
#plt.imshow(tmp, origin='lower')

In [16]:
fits.writeto(imagename.replace('.fits', '_clean.fits'), data=imarr_resid, header=imhdr, overwrite=True)